In [ ]:
%sh
pip install adal

Requirement already satisfied: adal in /databricks/python3/lib/python3.7/site-packages (1.2.7)
Requirement already satisfied: python-dateutil<3,>=2.1.0 in /databricks/python3/lib/python3.7/site-packages (from adal) (2.8.1)
Requirement already satisfied: PyJWT<3,>=1.0.0 in /databricks/python3/lib/python3.7/site-packages (from adal) (2.3.0)
Requirement already satisfied: requests<3,>=2.0.0 in /databricks/python3/lib/python3.7/site-packages (from adal) (2.22.0)
Requirement already satisfied: cryptography>=1.1.0 in /databricks/python3/lib/python3.7/site-packages (from adal) (2.8)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.7/site-packages (from python-dateutil<3,>=2.1.0->adal) (1.14.0)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests<3,>=2.0.0->adal) (1.25.8)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests<3,>=2.0.0->adal) (2.8)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests<3,>=2.0.0->adal) (2020.6.20)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /usr/lib/python3/dist-packages (from requests<3,>=2.0.0->adal) (3.0.4)
Requirement already satisfied: cffi!=1.11.3,>=1.8 in /databricks/python3/lib/python3.7/site-packages (from cryptography>=1.1.0->adal) (1.14.0)
Requirement already satisfied: pycparser in /databricks/python3/lib/python3.7/site-packages (from cffi!=1.11.3,>=1.8->cryptography>=1.1.0->adal) (2.19)
WARNING: You are using pip version 20.0.2; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python3.7 -m pip install --upgrade pip' command.

In [ ]:
# Import packages
import adal
import datetime

In [ ]:
# Set up database access
class sp_mssql_access:
    
    def __init__(self, tenant_id, authority, resource_app_id_url, service_principal_id, service_principal_secret, sql_hostname, database_name, database_port):
        
        self.tenant_id = tenant_id
        self.authority = authority
        self.resource_app_id_url = resource_app_id_url
        self.service_principal_id = service_principal_id
        self.service_principal_secret = service_principal_secret
        self.sql_hostname = sql_hostname
        self.database_name = database_name
        self.database_port = database_port
        self.jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(self.sql_hostname, self.database_port, self.database_name)
        self.token = None
        
    def _gen_token(self):
        context = adal.AuthenticationContext(self.authority)
        self.token = context.acquire_token_with_client_credentials(self.resource_app_id_url, self.service_principal_id, self.service_principal_secret)
        self.token['expiresOn'] = datetime.datetime.now() + datetime.timedelta(0, self.token['expiresIn'])
        
    def __getattribute__(self, name):
        if name == 'token':
            if super().__getattribute__(name) is None: self._gen_token()
            return super().__getattribute__(name)
        
        elif name == 'connectionProperties':
            if (self.token['expiresOn'] - datetime.timedelta(0, 1800)) <= datetime.datetime.now():
                self._gen_token()
            return {
                "accessToken": self.token['accessToken'],
                "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
            }    
        else:
            return super().__getattribute__(name)

In [ ]:
# connection arguments
connect_arg = {
  "user_tenant_id": "your_user_tenant_id",
  "user_authority": "your_user_authority_url",
  "user_resource_app_id_url": "https://database.windows.net/",
  "user_service_principal_id_scope": "your_user_service_principal_id_scope",
  "user_service_principal_id_key": "your_user_service_principal_id_key", 
  "user_service_principal_secret_scope": "your_user_service_principal_secret_scope",
  "user_service_principal_secret_key": "your_user_service_principal_secret_key",
  "user_sql_hostname": "your_user_sql_hostname",
  "user_database_name": "your_user_database_name",
  "user_database_port": "your_user_database_port"
}

In [ ]:
## db
db_access = sp_mssql_access(
    tenant_id = connect_arg.get('user_tenant_id'), 
    authority = connect_arg.get("user_authority"), 
    resource_app_id_url = connect_arg.get("user_resource_app_id_url"), 
    service_principal_id = dbutils.secrets.get(scope=connect_arg.get("user_service_principal_id_scope"), key=connect_arg.get("user_service_principal_id_key")), 
    service_principal_secret = dbutils.secrets.get(scope=connect_arg.get("user_service_principal_secret_scope"), key=connect_arg.get("user_service_principal_secret_key")), 
    sql_hostname = connect_arg.get("user_sql_hostname"), 
    database_name = connect_arg.get("user_database_name"), 
    database_port = connect_arg.get("user_database_port")
)

In [ ]:
# check accessible tables
pushdown_query = "(select * from INFORMATION_SCHEMA.TABLES) myalias"
accessible_table_list = spark.read\
.jdbc(
    url = db_access.jdbcUrl, 
    table=pushdown_query, 
    properties = db_access.connectionProperties
)
print('DB Connection is set up!')

SVoC Connection is set up!